In [64]:
# Importing important libraries
import pandas as pd
import numpy as np

# Import libaraies for tools
from sklearn.impute import SimpleImputer # For Handling missing values in data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import  ColumnTransformer

## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [54]:
data = pd.read_csv(r"F:\Project_template\notebooks\data\gemstone.csv",index_col='id')
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [55]:
# Data Preparation
X = data.drop('price',axis = 1)
Y = data["price"]
X.shape, Y.shape

((193573, 9), (193573,))

In [56]:
Categorical_columns = X.select_dtypes(include= 'object').columns
numerical_columns = X.select_dtypes(exclude="object").columns

In [57]:
cut_map = ['Fair', 'Good', 'Very Good','Premium','Ideal']
clarity_map = ["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"]
colore_map = ["D","E","F","G","H","I","J"]

In [58]:
# numerical pipeline
num_pipeline = Pipeline(steps= [
    ("imputer",SimpleImputer()),
    ("scalar",StandardScaler())
])

# Categorical pipeline
cat_pipeline = Pipeline(steps = [
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencodder',OrdinalEncoder(categories= [cut_map,colore_map,clarity_map]))
])

# Now implementing this pipelines
preprocessor = ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,numerical_columns),
        ('cat_pipeline',cat_pipeline,Categorical_columns)
    ]
)


In [59]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state=42,test_size=0.25)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape 

((145179, 9), (48394, 9), (145179,), (48394,))

In [60]:
preprocessor.fit_transform(X_train)

array([[ 0.47201886,  0.2579731 , -0.64043968, ...,  4.        ,
         5.        ,  1.        ],
       [ 0.62309937,  0.81246294, -1.16126779, ...,  4.        ,
         4.        ,  3.        ],
       [ 2.65189475, -1.86757127,  1.44287279, ...,  3.        ,
         5.        ,  3.        ],
       ...,
       [ 0.92526038,  0.90487791,  0.40121656, ...,  3.        ,
         3.        ,  3.        ],
       [-1.03878621, -0.66617663, -0.64043968, ...,  2.        ,
         3.        ,  6.        ],
       [-1.03878621, -0.01927182,  0.92204468, ...,  2.        ,
         3.        ,  1.        ]])

In [61]:
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [63]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.fit_transform(X_test), columns=preprocessor.get_feature_names_out())

In [65]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [67]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,Y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(Y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1018.3020227062416
MAE: 678.9281271436123
R2 score 93.61073317730339


Lasso
Model Training Performance
RMSE: 1018.3798348891911
MAE: 680.3583169705116
R2 score 93.60975668549084


Ridge
Model Training Performance
RMSE: 1018.3033898794762
MAE: 678.9588408931088
R2 score 93.61071602082082


Elasticnet
Model Training Performance
RMSE: 1513.1093453652181
MAE: 1056.612793124889
R2 score 85.89287743921612


